In [1]:
#imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pickle as pkl

In [2]:
classifier = pkl.load(open('rfc.pkl', 'rb'))
regressor = pkl.load(open('xgb_reg.pkl', 'rb'))

C:\Users\nithi\anaconda\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.25.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
type(classifier)

sklearn.ensemble._forest.RandomForestClassifier

In [4]:
type(regressor)

xgboost.sklearn.XGBRegressor

In [5]:
df = pd.read_csv('merged.csv', index_col = [0])

In [6]:
df.columns

Index(['FlightDate', 'Quarter', 'Year', 'Month', 'Origin', 'Dest',
       'DayofMonth', 'DepTime', 'DepDel15', 'CRSDepTime', 'DepDelayMinutes',
       'OriginAirportID', 'DestAirportID', 'ArrTime', 'CRSArrTime', 'ArrDel15',
       'ArrDelayMinutes', 'rounded_time', 'windspeedKmph', 'winddirDegree',
       'weatherCode', 'precipMM', 'visibility', 'pressure', 'cloudcover',
       'DewPointF', 'WindGustKmph', 'tempF', 'WindChillF', 'humidity', 'date',
       'time', 'airport_code'],
      dtype='object')

In [7]:
df = df.drop(columns = ['airport_code','rounded_time','date','FlightDate','OriginAirportID', 'DestAirportID','WindChillF', 'CRSArrTime', 'ArrTime', 'time', 'Quarter'])

In [8]:
df.columns

Index(['Year', 'Month', 'Origin', 'Dest', 'DayofMonth', 'DepTime', 'DepDel15',
       'CRSDepTime', 'DepDelayMinutes', 'ArrDel15', 'ArrDelayMinutes',
       'windspeedKmph', 'winddirDegree', 'weatherCode', 'precipMM',
       'visibility', 'pressure', 'cloudcover', 'DewPointF', 'WindGustKmph',
       'tempF', 'humidity'],
      dtype='object')

In [9]:
from sklearn.preprocessing import LabelEncoder
labelEncoder = LabelEncoder()
df["Origin"] = labelEncoder.fit_transform(df["Origin"])
df["Dest"] = labelEncoder.fit_transform(df["Dest"])

In [10]:
X = df.drop(columns = ['ArrDel15', 'ArrDelayMinutes'])

In [11]:
y = df[['ArrDel15', 'ArrDelayMinutes']]

In [12]:
y.head()

,ArrDel15,ArrDelayMinutes
0,0.0,8.0
1,1.0,24.0
2,0.0,0.0
3,0.0,10.0
4,0.0,0.0


In [15]:
from sklearn.metrics import accuracy_score,  mean_absolute_error, classification_report,  r2_score, mean_absolute_error, mean_squared_error

In [20]:
def run_models(X, y):
    #Classification
    y_pred = classifier.predict(X)
    print("Accuracy of Classification: ", accuracy_score(y_pred, y['ArrDel15']))
    print(classification_report(y_pred, y['ArrDel15']))
    df['y_pred'] = y_pred
    
    #Regression
    reg_df = df[df['y_pred'] == 1.0]
    #reg_df = reg_df.drop('y_pred', axis = 1)
    reg_df = reg_df[reg_df['ArrDelayMinutes'] > 15]
    X = reg_df.drop(columns = ['ArrDel15', 'ArrDelayMinutes', 'y_pred'])
    y = reg_df[['ArrDel15', 'ArrDelayMinutes']]
    ypred = regressor.predict(X)
    print("mae: ", mean_absolute_error(ypred, y['ArrDelayMinutes']))
    print("mse: ", mean_squared_error(ypred, y['ArrDelayMinutes']))
    mse = mean_squared_error(ypred, y['ArrDelayMinutes'])
    rmse = mse ** 0.5  
    print("RMSE: ", rmse)
    print("R2 score: ", r2_score(ypred, y['ArrDelayMinutes']))
    
    #reg_df.rename(columns = {'y_pred' : 'ArrDel15 (Predicted)'})
    reg_df['ArrDelayMinutes (Predicted)'] = ypred
    print(reg_df[['y_pred', 'ArrDelayMinutes (Predicted)']])
    #print(classification_report(ypred, y['ArrDelayMinutes']))
    return reg_df

In [21]:
result = run_models(X, y)

Accuracy of Classification:  0.9854451021967381
              precision    recall  f1-score   support

         0.0       0.98      1.00      0.99   1435942
         1.0       0.99      0.94      0.97    404332

    accuracy                           0.99   1840274
   macro avg       0.99      0.97      0.98   1840274
weighted avg       0.99      0.99      0.99   1840274

mae:  11.13045191725452
mse:  256.3153652072119
RMSE:  16.009852129461155
R2 score:  0.9490325601539928
         y_pred  ArrDelayMinutes (Predicted)
1           1.0                    33.487461
9           1.0                    36.690868
13          1.0                    32.461235
14          1.0                    28.116556
17          1.0                    24.995829
...         ...                          ...
1840262     1.0                    62.042576
1840264     1.0                    30.801378
1840265     1.0                    50.979843
1840271     1.0                    30.965372
1840272     1.0           

In [22]:
result[['y_pred', 'ArrDelayMinutes (Predicted)']]

,y_pred,ArrDelayMinutes (Predicted)
1,1.0,33.487461
9,1.0,36.690868
13,1.0,32.461235
14,1.0,28.116556
17,1.0,24.995829
...,...,...
1840262,1.0,62.042576
1840264,1.0,30.801378
1840265,1.0,50.979843
1840271,1.0,30.965372


In [23]:
result

,Year,Month,Origin,Dest,DayofMonth,DepTime,DepDel15,CRSDepTime,DepDelayMinutes,ArrDel15,...,precipMM,visibility,pressure,cloudcover,DewPointF,WindGustKmph,tempF,humidity,y_pred,ArrDelayMinutes (Predicted)
1,2016,1,13,14,1,759.0,1.0,724,35.0,1.0,...,0.0,10,1030,0,23,8,34,66,1.0,33.487461
9,2016,1,13,7,1,745.0,1.0,705,40.0,1.0,...,0.0,10,1030,0,23,8,34,66,1.0,36.690868
13,2016,1,13,8,2,720.0,1.0,655,25.0,1.0,...,0.0,10,1024,0,24,8,36,64,1.0,32.461235
14,2016,1,13,8,2,730.0,0.0,730,0.0,1.0,...,0.0,10,1024,0,24,8,36,64,1.0,28.116556
17,2016,1,13,0,2,745.0,0.0,745,0.0,1.0,...,0.0,10,1024,0,24,8,36,64,1.0,24.995829
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1840262,2017,9,6,8,21,2358.0,1.0,2245,73.0,1.0,...,0.0,10,1015,96,64,26,72,75,1.0,62.042576
1840264,2017,9,9,6,22,2021.0,1.0,1954,27.0,1.0,...,0.6,9,1013,20,73,24,79,80,1.0,30.801378
1840265,2017,9,9,4,22,2240.0,1.0,2140,60.0,1.0,...,0.0,9,1013,19,73,23,78,84,1.0,50.979843
1840271,2017,9,9,6,24,1948.0,0.0,1954,0.0,1.0,...,0.0,10,1012,14,71,19,82,71,1.0,30.965372


##### Regression Results Analysis

In [24]:
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error, accuracy_score
result['ArrDelayMinutes']

1          24.0
9          37.0
13         46.0
14         21.0
17         22.0
           ... 
1840262    51.0
1840264    18.0
1840265    57.0
1840271    38.0
1840272    44.0
Name: ArrDelayMinutes, Length: 368711, dtype: float64

In [25]:
#15 - 100
r1 = result[(result['ArrDelayMinutes'] >= 15) & (result['ArrDelayMinutes'] <= 100)]
mae = mean_absolute_error(r1['ArrDelayMinutes'], r1['ArrDelayMinutes (Predicted)'])
mse = mean_squared_error(r1['ArrDelayMinutes'], r1['ArrDelayMinutes (Predicted)'])
rmse = mse ** 0.5  
r2 = r2_score(r1['ArrDelayMinutes'], r1['ArrDelayMinutes (Predicted)'])

print("MAE: ", mae)
print("MSE: ", mse)
print("RMSE: ", rmse)
print("R-Squared: ", r2)


r1.shape

MAE:  9.969403665476527
MSE:  173.96946757787543
RMSE:  13.189748579024371
R-Squared:  0.6452788471150006


(305632, 24)

In [26]:
#100 - 200
r1 = result[(result['ArrDelayMinutes'] >= 100) & (result['ArrDelayMinutes'] <= 200)]
mae = mean_absolute_error(r1['ArrDelayMinutes'], r1['ArrDelayMinutes (Predicted)'])
mse = mean_squared_error(r1['ArrDelayMinutes'], r1['ArrDelayMinutes (Predicted)'])
rmse = mse ** 0.5  
r2 = r2_score(r1['ArrDelayMinutes'], r1['ArrDelayMinutes (Predicted)'])

print("MAE: ", mae)
print("MSE: ", mse)
print("RMSE: ", rmse)
print("R-Squared: ", r2)
r1.shape

MAE:  16.687774978729184
MSE:  628.6567379280857
RMSE:  25.073028096504135
R-Squared:  0.15534577457262966


(48867, 24)

In [27]:
#200 - 500
r1 = result[(result['ArrDelayMinutes'] >= 200) & (result['ArrDelayMinutes'] <= 500)]
mae = mean_absolute_error(r1['ArrDelayMinutes'], r1['ArrDelayMinutes (Predicted)'])
mse = mean_squared_error(r1['ArrDelayMinutes'], r1['ArrDelayMinutes (Predicted)'])
rmse = mse ** 0.5  
r2 = r2_score(r1['ArrDelayMinutes'], r1['ArrDelayMinutes (Predicted)'])

print("MAE: ", mae)
print("MSE: ", mse)
print("RMSE: ", rmse)
print("R-Squared: ", r2)
r1.shape

MAE:  17.155694847674283
MSE:  756.0141078055251
RMSE:  27.49571071650131
R-Squared:  0.8271231487171428


(14187, 24)

In [28]:
#500 - 1000
r1 = result[(result['ArrDelayMinutes'] >= 500) & (result['ArrDelayMinutes'] <= 1000)]
mae = mean_absolute_error(r1['ArrDelayMinutes'], r1['ArrDelayMinutes (Predicted)'])
mse = mean_squared_error(r1['ArrDelayMinutes'], r1['ArrDelayMinutes (Predicted)'])
rmse = mse ** 0.5  
r2 = r2_score(r1['ArrDelayMinutes'], r1['ArrDelayMinutes (Predicted)'])

print("MAE: ", mae)
print("MSE: ", mse)
print("RMSE: ", rmse)
print("R-Squared: ", r2)
r1.shape

MAE:  14.085746627917393
MSE:  444.2322071078242
RMSE:  21.076816816298997
R-Squared:  0.9775580913383243


(1112, 24)

In [29]:
#1000 - 2000
r1 = result[(result['ArrDelayMinutes'] >= 1000) & (result['ArrDelayMinutes'] <= 2000)]
mae = mean_absolute_error(r1['ArrDelayMinutes'], r1['ArrDelayMinutes (Predicted)'])
mse = mean_squared_error(r1['ArrDelayMinutes'], r1['ArrDelayMinutes (Predicted)'])
rmse = mse ** 0.5  
r2 = r2_score(r1['ArrDelayMinutes'], r1['ArrDelayMinutes (Predicted)'])

print("MAE: ", mae)
print("MSE: ", mse)
print("RMSE: ", rmse)
print("R-Squared: ", r2)
r1.shape

MAE:  14.78311421829841
MSE:  428.73953734536394
RMSE:  20.706026594819296
R-Squared:  0.9842589345348894


(173, 24)

In [30]:
new = pd.read_csv('merged.csv')

In [31]:
new.shape

(1840274, 34)